<a href="https://colab.research.google.com/github/louisowen6/NLP_Stacking_Ensemble/blob/master/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount the Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Import Packages

In [0]:
import warnings
warnings.filterwarnings("ignore")

#Basic
import pandas as pd
import gensim
import nltk
import re
import numpy as np
import math
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import rpy2.robjects.packages as rpackages

#For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#For Data Exploration
#Input the desired R packages
R_packages=['DataExplorer']
#Install the desired R packages
utils=rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)
for package in R_packages:
    if not(rpackages.isinstalled(package)):
        utils.install_packages(package)
#As a 'bridge' between python and R environment
r=robjects.r

#For Data Preparation
import itertools
from itertools import combinations

#For Class Definition
from sklearn.base import BaseEstimator, TransformerMixin

# Import Data

In [0]:
link ='https://drive.google.com/open?id=18QBSFqwHJ4-y5aAv_fcOd57pnNTUHjBo'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('df_prepared.csv')  
df = pd.read_csv('df_prepared.csv')
df=df.drop(['Unnamed: 0'],1)

18QBSFqwHJ4-y5aAv_fcOd57pnNTUHjBo


# Data Cleaning

## Basic

In [0]:
df.head(2)

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
0,$NFLX,NaN,0.000000,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],out $NFLX -.35,0.0,out $NFLX -.35,out $NFLX -.35,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0000,0.0,0.0000,0.0,0.0,0.0,-0.036883,0.000000,-0.036883,0.000,0.333333,0.0,NaN,NaN,0.0,0.0,4.275428,1.338285,NaN,NaN,NaN,NaN,NaN,NaN,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.343997
1,$PLUG,1.0,0.266667,"[0am,9am)",0.0,0.0,NaN,0.403,stocktwits,['Very intrigued with the technology and growt...,Been doing some work on $PLUG this evening. Ve...,1.0,Been doing some work on $PLUG this evening Ver...,been do some work on $PLUG thi even veri intri...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0625,0.0,0.0625,0.0,0.0,0.0,-0.204069,0.177081,-0.091274,0.375,0.562500,0.0,0.0,0.0,0.0,0.0,4.984655,1.432945,7.322638,1.224583,7.589614,1.098612,7.589614,1.098612,6,9,1,0,0,1,0,0,0,0,0,0,0,0,0,0.374176


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 89 columns):
cashtag                       1454 non-null object
conversation_parent           163 non-null float64
conversation_replies          919 non-null float64
created_at                    1454 non-null object
liked_by_self                 919 non-null float64
official_account              919 non-null float64
sentiment                     162 non-null object
sentiment score               1454 non-null float64
source                        1454 non-null object
spans                         1454 non-null object
text                          1454 non-null object
total_likes                   919 non-null float64
clean_text                    1454 non-null object
base_text                     1454 non-null object
POS_VB                        1454 non-null int64
POS_VBD                       1454 non-null int64
POS_VBG                       1454 non-null int64
POS_VBN                      

## Missing Values Analysis

In [0]:
def missing_values(df):
  '''
  Function to check features with missing values
  '''
  missing_values_feat=[]
  for column in df.columns:
    if df[column].isnull().values.any():
      missing_values_feat.append(column)
  return(missing_values_feat)

missing_values(df)

['conversation_parent',
 'conversation_replies',
 'liked_by_self',
 'official_account',
 'sentiment',
 'total_likes',
 'SentiWordNet_max_score',
 'SentiWordNet_min_score',
 'Avg_TFIDF_1-grams',
 'Avg_rf_1-grams',
 'Avg_TFIDF_2-grams',
 'Avg_rf_2-grams',
 'Avg_TFIDF_3-grams',
 'Avg_rf_3-grams',
 'Avg_TFIDF_4-grams',
 'Avg_rf_4-grams',
 'PMI_score']

In [0]:
len(df[df.source=='stocktwits'])

919

In [0]:
df[['conversation_parent']].describe()

,conversation_parent
count,163.000000
mean,0.723926
std,0.448431
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


Remove [conversation_parent], because too much missing values

In [0]:
df[['conversation_replies']].describe()

,conversation_replies
count,919.000000
mean,0.011643
std,0.052016
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, because the missing values is from twitter data

In [0]:
df[['liked_by_self']].describe()

,liked_by_self
count,919.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


Remove [liked_by_self], because the missing value is from twitter data

In [0]:
df[['official_account']].describe()

,official_account
count,919.000000
mean,0.132753
std,0.339492
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, because the missing value is from twitter data

In [0]:
df[['sentiment']].describe()

,sentiment
count,162
unique,2
top,Bullish
freq,123


In [0]:
df['sentiment'].unique()

array([nan, 'Bearish', 'Bullish'], dtype=object)

OHE: 1 for bullish -1 for bearish else 0, because the nan values assumed to be neutral

In [0]:
df[['total_likes']].describe()

,total_likes
count,919.000000
mean,0.046971
std,0.124470
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, becaue the missing data is from twitter data

In [0]:
df[['SentiWordNet_max_score']].describe()

,SentiWordNet_max_score
count,1430.000000
mean,0.232955
std,0.260000
min,-0.750000
25%,0.000000
50%,0.125000
75%,0.375000
max,1.000000


In [0]:
df[pd.isnull(df.SentiWordNet_max_score)].head()

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
0,$NFLX,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],out $NFLX -.35,0.0,out $NFLX -.35,out $NFLX -.35,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.036883,0.000000,-0.036883,0.000000,0.333333,0.0,NaN,NaN,0.0,0.0,4.275428,1.338285,NaN,NaN,NaN,NaN,NaN,NaN,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.343997
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.000000,-0.162826,0.000000,0.086957,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
172,$AAPL,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.270,stocktwits,['Out $AAPL'],Out $AAPL +1.2,0.0,Out $AAPL +1.2,out $AAPL +1.2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.108089,0.000000,-0.071206,0.000000,0.666667,0.0,NaN,NaN,0.0,0.0,4.275428,1.299283,NaN,NaN,NaN,NaN,NaN,NaN,31,34,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.421264
222,$SPY,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.114,stocktwits,['cover all'],cover all @ 119 +.2 $SPY,0.0,cover all @ 119 +.2 $SPY,cover all @ 119 +.2 $SPY,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.029548,0.064425,-0.071206,0.333333,0.500000,0.0,NaN,NaN,0.0,0.0,5.356660,1.546856,7.589614,1.098612,NaN,NaN,NaN,NaN,44,39,0,0,0,1,1,0,0,0,0,0,0,0,0,0.010876
277,$NEON,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.430,stocktwits,['oversold'],$NEON oversold,0.0,$NEON oversold,$NEON oversold,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0,0.0,6.673323,1.791759,NaN,NaN,NaN,NaN,NaN,NaN,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,1.650096


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 

In [0]:
df[['SentiWordNet_min_score']].describe()

,SentiWordNet_min_score
count,1430.000000
mean,-0.188199
std,0.258757
min,-1.000000
25%,-0.375000
50%,0.000000
75%,0.000000
max,0.500000


In [0]:
df[pd.isnull(df.SentiWordNet_min_score)].head()

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
0,$NFLX,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],out $NFLX -.35,0.0,out $NFLX -.35,out $NFLX -.35,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.036883,0.000000,-0.036883,0.000000,0.333333,0.0,NaN,NaN,0.0,0.0,4.275428,1.338285,NaN,NaN,NaN,NaN,NaN,NaN,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.343997
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.000000,-0.162826,0.000000,0.086957,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
172,$AAPL,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.270,stocktwits,['Out $AAPL'],Out $AAPL +1.2,0.0,Out $AAPL +1.2,out $AAPL +1.2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.108089,0.000000,-0.071206,0.000000,0.666667,0.0,NaN,NaN,0.0,0.0,4.275428,1.299283,NaN,NaN,NaN,NaN,NaN,NaN,31,34,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.421264
222,$SPY,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.114,stocktwits,['cover all'],cover all @ 119 +.2 $SPY,0.0,cover all @ 119 +.2 $SPY,cover all @ 119 +.2 $SPY,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.029548,0.064425,-0.071206,0.333333,0.500000,0.0,NaN,NaN,0.0,0.0,5.356660,1.546856,7.589614,1.098612,NaN,NaN,NaN,NaN,44,39,0,0,0,1,1,0,0,0,0,0,0,0,0,0.010876
277,$NEON,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.430,stocktwits,['oversold'],$NEON oversold,0.0,$NEON oversold,$NEON oversold,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.0,0.0,6.673323,1.791759,NaN,NaN,NaN,NaN,NaN,NaN,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,1.650096


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 

In [0]:
df[['Avg_TFIDF_1-grams']].describe()

,Avg_TFIDF_1-grams
count,1448.000000
mean,5.306872
std,0.795715
min,1.981975
25%,4.810385
50%,5.243674
75%,5.697248
max,15.179228


In [0]:
df[pd.isnull(df['Avg_TFIDF_1-grams'])]

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
291,$CAT,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.586,stocktwits,['$CAT +5.10%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
295,$X,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.144,stocktwits,['$X +2.27%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
761,$YHOO,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.160,stocktwits,['$YHOO +2.61%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
1230,$SBUX,NaN,NaN,"[9am,3pm)",NaN,NaN,NaN,-0.170,twitter,"['Deutsche Bank: cuts #Starbucks to Hold', 'pr...",e\n#JulieAnneSanJoseForSpottedPH,NaN,e #JulieAneSanJoseForSpotedPH,e #julieanesanjoseforspotedph,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.036883,0.0,-0.036883,0.0,0.500000,0.375,0.375,0.375,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
1302,$INTU,NaN,NaN,"[3pm,24pm)",NaN,NaN,NaN,0.161,twitter,['now a cloud software company. How the compan...,@_katrinaanicole 💖,NaN,💖,💖,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value

In [0]:
df[['Avg_rf_1-grams']].describe()

,Avg_rf_1-grams
count,1448.000000
mean,1.564321
std,0.266492
min,0.693147
25%,1.383244
50%,1.514660
75%,1.688176
max,3.460038


In [0]:
df[pd.isnull(df['Avg_rf_1-grams'])]

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
291,$CAT,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.586,stocktwits,['$CAT +5.10%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
295,$X,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.144,stocktwits,['$X +2.27%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
761,$YHOO,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.160,stocktwits,['$YHOO +2.61%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
1230,$SBUX,NaN,NaN,"[9am,3pm)",NaN,NaN,NaN,-0.170,twitter,"['Deutsche Bank: cuts #Starbucks to Hold', 'pr...",e\n#JulieAnneSanJoseForSpottedPH,NaN,e #JulieAneSanJoseForSpotedPH,e #julieanesanjoseforspotedph,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.036883,0.0,-0.036883,0.0,0.500000,0.375,0.375,0.375,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
1302,$INTU,NaN,NaN,"[3pm,24pm)",NaN,NaN,NaN,0.161,twitter,['now a cloud software company. How the compan...,@_katrinaanicole 💖,NaN,💖,💖,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Impute np.log(2) for all rf-ngram missing values, because the missing value is because no such word which pass the n-gram filter and np.log(2) is the minimum (neutral) value for rf-ngram

In [0]:
df[['PMI_score']].describe()

,PMI_score
count,1448.000000
mean,-0.167135
std,0.891885
min,-2.739231
25%,-0.821421
50%,0.080449
75%,0.503710
max,1.650096


In [0]:
df[pd.isnull(df.PMI_score)]

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,POS_VM,+num,-num,num%,+num%,-num%,$num,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,num/num/num,only_number,call_+num%,call_-num%,put_+num%,put_-num%,bull,...,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,General_Inquirer_pos_ratio,General_Inquirer_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
291,$CAT,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.586,stocktwits,['$CAT +5.10%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
295,$X,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.144,stocktwits,['$X +2.27%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
761,$YHOO,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.160,stocktwits,['$YHOO +2.61%'],"$CAT +5.10%, $RIO +4.54%, $FCX +3.53%, $FXI +2...",0.0,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,$CAT +5.10% $RIO +4.54% $FCX +3.53% $FXI +2.93...,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.237492,0.0,-0.162826,0.0,0.086957,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,10,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
1230,$SBUX,NaN,NaN,"[9am,3pm)",NaN,NaN,NaN,-0.170,twitter,"['Deutsche Bank: cuts #Starbucks to Hold', 'pr...",e\n#JulieAnneSanJoseForSpottedPH,NaN,e #JulieAneSanJoseForSpotedPH,e #julieanesanjoseforspotedph,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.036883,0.0,-0.036883,0.0,0.500000,0.375,0.375,0.375,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
1302,$INTU,NaN,NaN,"[3pm,24pm)",NaN,NaN,NaN,0.161,twitter,['now a cloud software company. How the compan...,@_katrinaanicole 💖,NaN,💖,💖,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,34,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Impute 0 for missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value

## Missing Values Imputation

Missing Value Actionable Items:
1.   Remove [conversation_parent], because too much missing values
2.   Impute -1 for [conversation_replies] missing values, because the missing values is from twitter data
3.   Remove [liked_by_self], because constant feature
4.   Impute -1 for [official_account] missing values, because the missing value is from twitter data
5.   OHE [sentiment]: 1 for bullish -1 for bearish else 0, because the nan values assumed to be neutral
6.   Impute -1 for [total_likes] missing values, becaue the missing data is from twitter data
7.   Impute 0 for [SentiWordNet_max_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
8.   Impute 0 for [SentiWordNet_min_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
9.   Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value
10. Impute np.log(2) for all rf-ngram missing values, because the missing value is because no such word which pass the n-gram filter and np.log(2) is the minimum (neutral) value for rf-ngram
11. Impute 0 for [PMI_score] missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value






In [0]:
df=df.drop(['conversation_parent','liked_by_self'],1)
df['conversation_replies']=df['conversation_replies'].fillna(-1)
df['official_account']=df['official_account'].fillna(-1)
df['sentiment']=df['sentiment'].apply(lambda x: 1 if x=='Bullish' else -1 if x=='Bearish' else 0)
df['total_likes']=df['total_likes'].fillna(-1)
df['SentiWordNet_max_score']=df['SentiWordNet_max_score'].fillna(0)
df['SentiWordNet_min_score']=df['SentiWordNet_min_score'].fillna(0)
df['Avg_TFIDF_1-grams']=df['Avg_TFIDF_1-grams'].fillna(0)
df['Avg_TFIDF_2-grams']=df['Avg_TFIDF_2-grams'].fillna(0)
df['Avg_TFIDF_3-grams']=df['Avg_TFIDF_3-grams'].fillna(0)
df['Avg_TFIDF_4-grams']=df['Avg_TFIDF_4-grams'].fillna(0)
df['Avg_rf_1-grams']=df['Avg_rf_1-grams'].fillna(np.log(2))
df['Avg_rf_2-grams']=df['Avg_rf_2-grams'].fillna(np.log(2))
df['Avg_rf_3-grams']=df['Avg_rf_3-grams'].fillna(np.log(2))
df['Avg_rf_4-grams']=df['Avg_rf_4-grams'].fillna(np.log(2))
df['PMI_score']=df['PMI_score'].fillna(0)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 87 columns):
cashtag                       1454 non-null object
conversation_replies          1454 non-null float64
created_at                    1454 non-null object
official_account              1454 non-null float64
sentiment                     1454 non-null int64
sentiment score               1454 non-null float64
source                        1454 non-null object
spans                         1454 non-null object
text                          1454 non-null object
total_likes                   1454 non-null float64
clean_text                    1454 non-null object
base_text                     1454 non-null object
POS_VB                        1454 non-null int64
POS_VBD                       1454 non-null int64
POS_VBG                       1454 non-null int64
POS_VBN                       1454 non-null int64
POS_VBP                       1454 non-null int64
POS_VBZ                     

## Constant Features

In [0]:
class remove_constant_features(BaseEstimator,TransformerMixin):
  '''
  Class for removing constant features either categorical or numeric features
  
  Tolerance==0.01 means the numeric features is 99% constant
  '''
  def __init__(self,tolerance,verbose=False):
    self.verbose=verbose
    self.tolerance=tolerance
  
  def fit(self,df):
    self.df=df
    self.num_constant_features=[]
    self.cat_constant_features=[]
    columns=self.df.columns.tolist()
    for column in columns:
      # check constant features for numerical columns
      if np.issubdtype(self.df[column].dtype, np.number):
        if self.df[column].std()<=self.tolerance:
          self.num_constant_features.append(column)
      else:
        try:
          # check constant features for categorical columns
          if len(self.df[feat].unique())== 1:
            self.cat_constant_features.append(column)
        except:
          None
    return(self)
  
  def transform(self,df):
    self.df=self.df.drop(self.num_constant_features,1)
    self.df=self.df.drop(self.cat_constant_features,1)
    if self.verbose:
      print('Removed Features: ',self.num_constant_features+self.cat_constant_features)
      print('')
    return(self.df)

In [0]:
constant=remove_constant_features(tolerance=0.01,verbose=True)
df=constant.fit_transform(df.copy())

Removed Features:  ['POS_VM', '$num', 'num/num/num', 'call_-num%', 'General_Inquirer_pos_ratio', 'General_Inquirer_neg_ratio']



In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 81 columns):
cashtag                   1454 non-null object
conversation_replies      1454 non-null float64
created_at                1454 non-null object
official_account          1454 non-null float64
sentiment                 1454 non-null int64
sentiment score           1454 non-null float64
source                    1454 non-null object
spans                     1454 non-null object
text                      1454 non-null object
total_likes               1454 non-null float64
clean_text                1454 non-null object
base_text                 1454 non-null object
POS_VB                    1454 non-null int64
POS_VBD                   1454 non-null int64
POS_VBG                   1454 non-null int64
POS_VBN                   1454 non-null int64
POS_VBP                   1454 non-null int64
POS_VBZ                   1454 non-null int64
+num                      1454 non-null int64
-num

## Duplicated Features

In [0]:
class remove_duplicated_features(BaseEstimator,TransformerMixin):
  '''
  Class for removing duplicated features
  '''
  def __init__(self,verbose=False):
    self.verbose=verbose
    
  def fit(self,df):
    self.df=df
    self.duplicated_feat = []
    for i in range(0, len(self.df.columns)):
      col_1 = self.df.columns[i]
      
      for col_2 in self.df.columns[i + 1:]:
        # if the features are duplicated
        if self.df[col_1].equals(self.df[col_2]):
            self.duplicated_feat.append(col_2)
    return(self)
  
  def transform(self,df):
    self.df=self.df.drop(self.duplicated_feat,1)
    if self.verbose:
      print('Removed Features: ',self.duplicated_feat)
      print('')
    return(self.df)

In [0]:
duplic=remove_duplicated_features(verbose=True)
df=duplic.fit_transform(df.copy())

Removed Features:  []



## Encoding

In [0]:
df['created_at']=df['created_at'].apply(lambda x: 0 if x=='[0am,9am)' else 1 if x=='[9am,3pm)' else 2)

## Correlated Features


In [0]:
def correlated(df,tolerance):
  '''
  Function to know correlated variables
  Input: {dataframe, correlation_tolerance}
  '''
  cate_features_index = np.where(df.dtypes == 'O')[0]
  num_features_index = [x for x in range(len(df.columns)) if x not in cate_features_index]
  
  #Check correlation of numerical variables
  corrmat = df.iloc[:,num_features_index].corr()
  corrmat = corrmat.abs().unstack() # absolute value of corr coef
  corrmat = corrmat.sort_values(ascending=False)
  corrmat = corrmat[corrmat >= tolerance]
  corrmat = corrmat[corrmat < 1]
  corrmat = pd.DataFrame(corrmat).reset_index()
  corrmat.columns = ['feature1', 'feature2', 'corr']

  # find groups of correlated features

  grouped_feature_ls = []
  correlated_groups = []

  for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:
      # find all features correlated to a single feature
      correlated_block = corrmat[corrmat.feature1 == feature]
      grouped_feature_ls = grouped_feature_ls + list(
      correlated_block.feature2.unique()) + [feature]

      # append the block of features to the list
      correlated_groups.append(correlated_block)

  print('found {} correlated groups'.format(len(correlated_groups)))
  print('out of {} total features'.format(df.iloc[:,num_features_index].shape[1]))

  for group in correlated_groups:
    print(group)
    print()

In [0]:
correlated(df.drop('source',1),0.9)

found 4 correlated groups
out of 75 total features
      feature1              feature2      corr
0  total_likes  conversation_replies  0.980199

         feature1        feature2      corr
2  Avg_rf_2-grams  Avg_rf_3-grams  0.936827

      feature1 feature2      corr
4  NER_PERCENT     num%  0.918903

         feature1        feature2      corr
6  Avg_rf_4-grams  Avg_rf_3-grams  0.904372



Remove conversation_replies, num%

In [0]:
df=df.drop(['conversation_replies','num%'],1)

## Final Touch

In [0]:
df['spans']=df['spans'].apply(lambda x: x[2:-2])

# Data Report

In [0]:
df.head(2)

,cashtag,created_at,official_account,sentiment,sentiment score,spans,text,total_likes,clean_text,base_text,POS_VB,POS_VBD,POS_VBG,POS_VBN,POS_VBP,POS_VBZ,+num,-num,+num%,-num%,word_num,ordinal_num,num-num,num-num%,num-num-num,num/num,only_number,call_+num%,put_+num%,put_-num%,bull,bear,number_of_!,number_of_?,number_of_$,continous_!,continous_?,AFINN_sum_score,AFINN_max_score,AFINN_min_score,AFINN_pos_ratio,AFINN_neg_ratio,BingLiu_pos_ratio,BingLiu_neg_ratio,NRC_Hashtag_sum_score,NRC_Hashtag_max_score,NRC_Hashtag_min_score,NRC_Hashtag_pos_ratio,NRC_Hashtag_neg_ratio,SentiWordNet_sum_score,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_rf_1-grams,Avg_TFIDF_2-grams,Avg_rf_2-grams,Avg_TFIDF_3-grams,Avg_rf_3-grams,Avg_TFIDF_4-grams,Avg_rf_4-grams,W2V_50_Means,GloVe40_Means,NER_DATE,NER_DURATION,NER_LOCATION,NER_MONEY,NER_NUMBER,NER_ORDINAL,NER_ORGANIZATION,NER_PERCENT,NER_PERSON,NER_SET,NER_TIME,caps_word,hashtags,PMI_score
0,$NFLX,2,0.0,0,-0.494,out $NFLX -.35,out $NFLX -.35,0.0,out $NFLX -.35,out $NFLX -.35,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,-0.036883,0.000000,-0.036883,0.000,0.333333,0.0,0.0,0.0,0.0,0.0,4.275428,1.338285,0.000000,0.693147,0.000000,0.693147,0.000000,0.693147,26,3,0,0,0,1,0,0,0,0,0,0,0,0,0,-0.343997
1,$PLUG,0,0.0,0,0.403,Very intrigued with the technology and growth ...,Been doing some work on $PLUG this evening. Ve...,1.0,Been doing some work on $PLUG this evening Ver...,been do some work on $PLUG thi even veri intri...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.4,0.4,0.0,0.0625,0.0,0.0625,0.0,-0.204069,0.177081,-0.091274,0.375,0.562500,0.0,0.0,0.0,0.0,0.0,4.984655,1.432945,7.322638,1.224583,7.589614,1.098612,7.589614,1.098612,6,9,1,0,0,1,0,0,0,0,0,0,0,0,0,0.374176


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 78 columns):
cashtag                   1454 non-null object
created_at                1454 non-null int64
official_account          1454 non-null float64
sentiment                 1454 non-null int64
sentiment score           1454 non-null float64
spans                     1454 non-null object
text                      1454 non-null object
total_likes               1454 non-null float64
clean_text                1454 non-null object
base_text                 1454 non-null object
POS_VB                    1454 non-null int64
POS_VBD                   1454 non-null int64
POS_VBG                   1454 non-null int64
POS_VBN                   1454 non-null int64
POS_VBP                   1454 non-null int64
POS_VBZ                   1454 non-null int64
+num                      1454 non-null int64
-num                      1454 non-null int64
+num%                     1454 non-null int64
-num%   

In [0]:
def create_report(df,target=None):
  '''
  Function to generate compact Data Analysis report
  '''
  DataExplorer=rpackages.importr('DataExplorer')
  robjects.globalenv['df']=pandas2ri.py2ri(df)
  robjects.globalenv['target']=target
  if target!=None:
    r('''
    config <- configure_report(
    add_plot_str = TRUE,
    add_plot_qq = TRUE,
    add_plot_prcomp = FALSE,
    add_plot_boxplot = FALSE,
    add_plot_scatterplot = TRUE,
    global_ggtheme = quote(theme_minimal(base_size = 14)))
    create_report(df,y=target,config=config)
    ''')
  else:
    r('create_report(df)')
    
target='sentiment.score'
#target='price'

create_report(df,target=target)

  |                                                                       
  |                                                                 |   0%
  |                                                                       
  |..                                                               |   2%


   
inline R code fragments



  |                                                                       
  |...                                                              |   5%


label:
 
global_options
 (with options) 

List

 of 

1



 

$ 

include

:
 logi 

FALSE





  |                                                                       
  |.....                                                            |   7%


  ordinary text without R code



  |                                                                       
  |......                                                           |  10%


label:
 
introduce


  |                                                         

# Data Split

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['sentiment score'],1), df['sentiment score'],stratify=df['source'],test_size=0.2)

In [0]:
X_train['sentiment score']=y_train
X_test['sentiment score']=y_test
X_train=X_train.reset_index()
X_test=X_test.reset_index()

# Export

In [0]:
X_train.to_csv('df_train_final.csv')
X_test.to_csv('df_test_final.csv')